## mzML Format test

In [ ]:
import base64, struct

In [ ]:
def base64_to_float(in_base64_str, float_bit=64):
    dec_str = base64.decodebytes(in_base64_str)
    
    if(float_bit==64):
        dec_float = struct.unpack('<%sd' % (len(dec_str) // 8), dec_str) # for 64 bit data
    else:
        dec_float = struct.unpack('<%sf' % (len(dec_str) // 4), dec_str) # for 32 bit data
    
    return dec_float

In [ ]:
mz_enc = b"AAAAAAAAAAAAAAAAAADwPwAAAAAAAABAAAAAAAAACEAAAAAAAAAQQAAAAAAAABRAAAAAAAAAGEAAAAAAAAAcQAAAAAAAACBAAAAAAAAAIkA="
intensity_enc = b"AAAAAAAAJEAAAAAAAAAiQAAAAAAAACBAAAAAAAAAHEAAAAAAAAAYQAAAAAAAABRAAAAAAAAAEEAAAAAAAAAIQAAAAAAAAABAAAAAAAAA8D8="

mz = base64_to_float(mz_enc)
print(mz)

intensity = base64_to_float(intensity_enc)
print(intensity)

In [ ]:
mz_enc = b"AAAAAAAAAAAAAAAAAADwPwAAAAAAAABAAAAAAAAACEAAAAAAAAAQQAAAAAAAABRAAAAAAAAAGEAAAAAAAAAcQAAAAAAAACBAAAAAAAAAIkAAAAAAAAAkQAAAAAAAACZAAAAAAAAAKEAAAAAAAAAqQAAAAAAAACxAAAAAAAAALkAAAAAAAAAwQAAAAAAAADFAAAAAAAAAMkAAAAAAAAAzQA=="
intensity_enc = b"AAAAAAAANEAAAAAAAAAzQAAAAAAAADJAAAAAAAAAMUAAAAAAAAAwQAAAAAAAAC5AAAAAAAAALEAAAAAAAAAqQAAAAAAAAChAAAAAAAAAJkAAAAAAAAAkQAAAAAAAACJAAAAAAAAAIEAAAAAAAAAcQAAAAAAAABhAAAAAAAAAFEAAAAAAAAAQQAAAAAAAAAhAAAAAAAAAAEAAAAAAAADwPw=="

mz = base64_to_float(mz_enc)
print(mz)

intensity = base64_to_float(intensity_enc)
print(intensity)

# MGF foramt analysis

In [7]:
import pyteomics.mgf
import os

In [8]:
mgf_path = '/home/workspace/MS-Tools/PXD001197/'

mgf_file = 'OrbiElite04549.mgf'

file_list = list()
file_list = file_list + [mgf_path + mgf_file]

In [ ]:
print(file_list)

In [9]:
f_in = pyteomics.mgf.MGF(mgf_path+mgf_file)
filename = os.path.splitext(os.path.basename(f_in.name))[0]
for spectrum_i, spectrum_dict in enumerate(f_in):
    print(spectrum_i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
print(spectrum_dict)

In [10]:
print(spectrum_dict['params'])

print(spectrum_dict['params']['title'])

print(spectrum_dict['params']['pepmass'][0])

print(spectrum_dict['params']['charge'])

# print(spectrum_dict['m/z array'])
# print(spectrum_dict['intensity array'])


{'title': 'controllerType=0 controllerNumber=1 scan=885', 'scans': '885', 'rtinseconds': 195.3685, 'pepmass': (730.829223632813, None), 'charge': [2]}
controllerType=0 controllerNumber=1 scan=885
730.829223632813
2+


In [ ]:
import numpy as np 
def test():
    quantization_error = np.zeros(2)
    quantization_error[0] = quantization_error[0] + 1
    quantization_error[1] = quantization_error[1] + 2
    return quantization_error

a = test()

In [16]:
a = {'b':1}
del a['b']

## Preprocessing for MGF files

In [1]:
import os
import sys
import json
import logging
import pyteomics.mgf
import numpy as np
from joblib import Parallel, delayed

logger = logging.getLogger('ms_preprocess_fp')

def run_baseline(**kargs):
    # Configure logging.
#     logging.captureWarnings(True)
#     root = logging.getLogger()
#     root.setLevel(logging.DEBUG)
#     handler = logging.StreamHandler(sys.stderr)
#     handler.setLevel(logging.DEBUG)
#     handler.setFormatter(logging.Formatter(
#         '{asctime} {levelname} [{name}/{processName}] {module}.{funcName} : '
#         '{message}', style='{'))
#     root.addHandler(handler)

    print(kargs)
    
    ############### Preprocess spectra ###############
    logger.info("Start processsing spectra")
    results = Parallel(n_jobs=-1, verbose=100)(delayed(preprocess_spectrum)(file_i, **kargs) for file_i in kargs['file_list'])
    logger.info("End processsing spectra")
    
    ############### Merge and reduce results ###############
    logger.info("Merging spectra results")    
    merged_processed_spectra = {}
    total_quantization_error = np.zeros(2)

    for result_i in results:
        ############### Gather spectra results ###############
        spec_i = result_i[0]
        if not merged_processed_spectra:
            merged_processed_spectra = spec_i
        else:
            for key in spec_i:
                if key not in merged_processed_spectra:
                    merged_processed_spectra[key] = spec_i[key]
                else:
                    merged_processed_spectra[key] = merged_processed_spectra[key] + spec_i[key]

        ############### Gather quantization error ###############
        quan_err_i = result_i[1]
        total_quantization_error = total_quantization_error+quan_err_i

    merged_processed_spectra['quantization_error'] = (total_quantization_error/len(results)).tolist()
    logger.info("Merging spectra results complete!")
    
    # Dump preprocessed results
    logger.info("Exporting results")
    with open(kargs['out_file_name']+'_parms.json', 'w') as convert_file:
        kargs['fp_quantizer_mz'] = None
        kargs['fp_quantizer_peak'] = None
        convert_file.write(json.dumps(kargs))
    with open(kargs['out_file_name']+'_data.json', 'w') as convert_file:
        convert_file.write(json.dumps(merged_processed_spectra))
    
    mz_error = merged_processed_spectra['quantization_error'][0]
    peak_error = merged_processed_spectra['quantization_error'][1]
    logger.info("All finished! With quantization error: %f %f", mz_error, peak_error)
    
    return merged_processed_spectra
    
    
def preprocess_spectrum(mgf_file, **kargs):
    f_in = pyteomics.mgf.MGF(mgf_file)
    filename = os.path.splitext(os.path.basename(f_in.name))[0]

    logger.info('Processing mgf file: %s', filename)
    processed_spectra_list = {}
    quantization_error = np.zeros(2)
    for spectrum_i, spectrum_dict in enumerate(f_in):
        spec_i_charge = str(spectrum_dict['params']['charge'])
        spec_i_title = spectrum_dict['params']['title']
        spec_i_precursor_mz = spectrum_dict['params']['pepmass'][0]

        spec_i_mz = spectrum_dict['m/z array'].astype(np.float32)
        spec_i_intensity = spectrum_dict['intensity array'].astype(np.float32)
        
#         print('maximum intensity: ', spec_i_intensity.max())
        if kargs['if_fp_quantize']:
            quantized_spec_i_mz = kargs['fp_quantizer_mz'].quantize(spec_i_mz)
            quantized_spec_i_intensity = kargs['fp_quantizer_peak'].quantize(spec_i_intensity)
            
            quantization_error[0] = quantization_error[0] + np.absolute(np.subtract(quantized_spec_i_mz, spec_i_mz)).mean()   
            quantization_error[1] = quantization_error[1] + np.absolute(np.subtract(quantized_spec_i_intensity, spec_i_intensity)).mean()
            
            (spec_i_mz, spec_i_intensity) = (quantized_spec_i_mz, quantized_spec_i_intensity)

        # 1. Filter spectrum out of mz range + low-intensity peaks
        spec_mask = filter_mz_intensity_range(spec_i_mz, spec_i_intensity, kargs['mz_min'], kargs['mz_max'], kargs['min_intensity'])
        (spec_i_mz , spec_i_intensity) = (spec_i_mz[spec_mask], spec_i_intensity[spec_mask])
        if not _check_spectrum_valid(spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
            continue

        # 2. Remove peaks related to precursor
        if kargs['remove_precursor_tol'] is not None:
            spec_mask = _remove_precursor_peak(spec_i_mz, kargs['remove_precursor_tol'], spec_i_precursor_mz)
            (spec_i_mz , spec_i_intensity) = (spec_i_mz[spec_mask], spec_i_intensity[spec_mask])

            if not _check_spectrum_valid(spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
                continue

        # 3. Remove low-intensity peaks
        if kargs['max_peaks_used'] is not None:
            max_peaks_used = kargs['max_peaks_used']

            spec_mask = _select_top_intensity(spec_i_intensity, max_peaks_used)
            (spec_i_mz , spec_i_intensity) = (spec_i_mz[spec_mask], spec_i_intensity[spec_mask])

            if not _check_spectrum_valid(spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
                continue

        # 4. Sclae and normalize peaks
        spec_i_intensity = _scale_intensity(spec_i_intensity, kargs['scaling'])
        spec_i_intensity = _norm_intensity(spec_i_intensity)

        if spec_i_charge not in processed_spectra_list:
            processed_spectra_list[spec_i_charge] = []

        processed_spectra_list[spec_i_charge] = processed_spectra_list[spec_i_charge] + [spec_i_title]

    return processed_spectra_list, quantization_error


def filter_mz_intensity_range(spectrum_mz: np.ndarray, spectrum_intensity: np.ndarray, mz_min: float, mz_max: float, min_intensity: float = 0.0,):
    """
        Filter spectrum out of mz range and low-intensity peaks

        Parameters
        ----------
        spectrum_mz : np.ndarray
            M/z peaks of the cluster whose quality is checked.
        spectrum_intensity: np.ndarray
            Intensity of the cluster whose quality is checked.
        mz_min : float
            Minimum number of peaks the cluster has to contain.
        mz_max : float
            Minimum number of peaks the cluster has to contain.
        min_intensity : float
            Remove peaks whose intensity is below `min_intensity` percentage
            of the intensity of the most intense peak (the default is 0, which
            means that no minimum intensity filter will be applied).
        Returns
        -------
        np.ndarray
            Index mask specifying which peaks are retained after intensity filtering
        """
    mask_mz = (spectrum_mz >= mz_min) & (spectrum_mz <= mz_max)
    mask_intensity = spectrum_intensity >= min_intensity
    return mask_mz & mask_intensity

    
def _check_spectrum_valid(spectrum_mz: np.ndarray, min_peaks: int, min_mz_range: float) -> bool:
    """
    Check whether a cluster is of good enough quality to be used.
    Parameters
    ----------
    spectrum_mz : np.ndarray
        M/z peaks of the cluster whose quality is checked.
    min_peaks : int
        Minimum number of peaks the cluster has to contain.
    min_mz_range : float
        Minimum m/z range the cluster's peaks need to cover.
    Returns
    -------
    bool
        True if the cluster has enough peaks covering a wide enough mass
        range, False otherwise.
    """
    return (len(spectrum_mz) >= min_peaks and spectrum_mz[-1] - spectrum_mz[0] >= min_mz_range)   


def _remove_precursor_peak(spectrum_mz: np.ndarray, precursor_tolerance: float, precursor_mz: float):
    """
    Remove peaks realted to the precursor
    Parameters
    ----------
    spectrum_mz : np.ndarray
        M/z peaks of the cluster to be processed.
    precursor_tolerance : float
        Fragment mass tolerance around the precursor mass to remove the precursor peak.
    precursor_mz : float
        Precursor mz.
    Returns
    -------
    np.ndarray
        Index mask specifying which peaks are retained after precursor peak
        filtering
    """
    mask = (spectrum_mz<=np.absolute(precursor_mz-precursor_tolerance)) | (spectrum_mz>=np.absolute(precursor_mz+precursor_tolerance))
    return mask


def _select_top_intensity(spectrum_intensity: np.ndarray, max_num_peaks: int = 50):
        """
        Only the `max_num_peaks` most intense fragment peaks are retained.

        Parameters
        ----------
        max_num_peaks : Optional[int], optional
            Only retain the `max_num_peaks` most intense peaks (the default is
            None, which retains all peaks).

        Returns
        -------
        np.ndarray
            Index mask specifying which peaks are retained after intensity filtering
        """
        intensity_idx = np.argsort(spectrum_intensity)
        
        # Only retain at most the `max_num_peaks` most intense peaks.
        mask = np.full_like(spectrum_intensity, False, np.bool_)
        mask[intensity_idx[-min(len(intensity_idx), max_num_peaks):]] = True
        return mask
    
    
def _scale_intensity(spectrum_intensity: np.ndarray, scaling: str = None, max_intensity: float = None):
    """
        Scale the intensity of the fragment peaks.

        Two types of scaling can be performed: scaling all peaks using a
        specific transformation and scaling the peaks relative to the most
        intense peak.

        Parameters
        ----------
        scaling : {'root', 'log'}, optional
            Method to scale the peak intensities (the default is None, which
            means that no transformation will be performed).
            Potential transformation options are:
            
            - 'root': Root-transform the peak intensities. The default is a
              square root transformation (`degree` is 2). The degree of the
              root can be specified using the `degree` kwarg.
            - 'log':  Log-transform the peak intensities. The default is a log2
              transformation (`base` is 2) after summing the intensities with 1
              to avoid negative values after the transformation. The base of
              the logarithm can be specified using the `base` kwarg.

        max_intensity : Optional[float], optional
            Intensity of the most intense peak relative to which the peaks will
            be scaled (the default is None, which means that no scaling
            relative to the most intense peak will be performed).

        Returns
        -------
        self : `MsmsSpectrum`
    """
    if scaling == 'root':
        spectrum_intensity = np.sqrt(spectrum_intensity).astype(np.float32)
    elif scaling == 'log':
        base = 2
        spectrum_intensity = (np.log1p(spectrum_intensity) / np.log(base)).astype(np.float32)
    
    if max_intensity is not None:
        spectrum_intensity = spectrum_intensity * max_intensity / spectrum_intensity.max()
        
    return spectrum_intensity


def _norm_intensity(spectrum_intensity: np.ndarray, scale: float = 1.0):
    """
    Normalize cluster peak intensities by their vector norm.
    Parameters
    ----------
    spectrum_intensity : np.ndarray
        The cluster peak intensities to be normalized.
    Returns
    -------
    np.ndarray
        The normalized peak intensities.
    """
    return spectrum_intensity / spectrum_intensity.max() * scale

In [2]:
def get_spec_stat(spectra_dict):
    total_spec = 0
    print('Spectrum statistics:')
    for charge_i in spectra_dict.keys():
        print(charge_i, len(spectra_dict[charge_i]))
        total_spec += len(spectra_dict[charge_i])

    print('Total spectrum: ', total_spec)

In [6]:
import glob

mgf_path = '/home/workspace/MS-Tools/PXD001511/'
mgf_file_list = glob.glob(mgf_path+'*.mgf')

# mgf_file_list = mgf_file_list[0:8]
# print(mgf_file_list)

out_path = './json_output/'
out_file_name = 'fp_baseline'
out_file_name = out_path + out_file_name

args = {
    'file_list': mgf_file_list, 
    'mz_min': 200, 
    'mz_max': 2000, 
    'min_mz_range': 250, 
    'min_peaks': 5, 
    'max_peaks_used': 50, 
    'remove_precursor_tol': 1.5, 
    'min_intensity': 0.01, 
    'scaling': 'log', 
    'out_file_name': out_file_name,
    'if_fp_quantize': False
}
processed_spectra_dict = run_baseline(**args)

{'file_list': ['/home/workspace/MS-Tools/PXD001511/A13-01201.mgf', '/home/workspace/MS-Tools/PXD001511/A13-03235.mgf', '/home/workspace/MS-Tools/PXD001511/A15-03064.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01202.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01203.mgf', '/home/workspace/MS-Tools/PXD001511/A13-03239.mgf', '/home/workspace/MS-Tools/PXD001511/A15-03065.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01210.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01211.mgf', '/home/workspace/MS-Tools/PXD001511/A13-03240.mgf', '/home/workspace/MS-Tools/PXD001511/A15-03067.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01212.mgf', '/home/workspace/MS-Tools/PXD001511/A13-03243.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01214.mgf', '/home/workspace/MS-Tools/PXD001511/A15-03068.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01215.mgf', '/home/workspace/MS-Tools/PXD001511/A13-01216.mgf', '/home/workspace/MS-Tools/PXD001511/A13-03244.mgf', '/home/workspace/MS-Tools/PXD001511/A15-03069.mgf

## Quantizer for MS Preprocessing

In [10]:
class ms_quantizer():
    def __init__(self, convert_exp=5, convert_frac=10):
        self.convert_exp = convert_exp
        self.convert_frac = convert_frac
        
        self.fp32_frac_bit = 23
        self.fp32_exp_bit = 8

        self.fp64_frac_bit = 52
        self.fp64_exp_bit = 11

        # For fp32
        self.offset_fp32 = 127-2**(self.convert_exp-1)+1
        self.mask_fp32 = np.array(0).astype(np.int32)            
        for i in range(self.fp32_frac_bit-self.convert_frac, self.fp32_frac_bit+self.convert_exp, 1):
            self.mask_fp32 = (self.mask_fp32 | np.array(1).astype(np.int32)<<i).astype(np.int32)
        
        # For fp64
        self.offset_fp64 = 1023-2**(self.convert_exp-1)+1
        self.mask_fp64 = np.array(0).astype(np.int64)            
        for i in range(self.fp64_frac_bit-self.convert_frac, self.fp64_frac_bit+self.convert_exp, 1):
            self.mask_fp64 = (self.mask_fp64 | np.array(1).astype(np.int64)<<i).astype(np.int64)
    
    def quantize(self, num):
        if num.dtype == np.float32:
            num = num * (np.array(2**self.offset_fp32).astype(np.float32))
            conveted_num = np.bitwise_and(num.view(np.int32), self.mask_fp32)
            conveted_num = conveted_num.view(np.float32) * (2**self.offset_fp32)
        else:
            num = num * (np.array(2**self.offset_fp64).astype(np.float64))
            conveted_num = np.bitwise_and(num.view(np.int64), self.mask_fp64)
            conveted_num = conveted_num.view(np.float64) * (2**self.offset_fp64)
    
        return conveted_num.astype(num.dtype)

In [18]:
import glob

mgf_path = '/home/workspace/MS-Tools/PXD001197/'
mgf_file_list = glob.glob(mgf_path+'*.mgf')

# mgf_file_list = mgf_file_list[0:8]


# Using fp16 quantization
mz_exp_bit = 4
peak_exp_bit = 6

mz_frac_bit = 16-mz_exp_bit
peak_frac_bit = 16-peak_exp_bit

fp16_quantizer_mz = ms_quantizer(convert_exp=mz_exp_bit+1, convert_frac=mz_frac_bit)
fp16_quantizer_peak = ms_quantizer(convert_exp=peak_exp_bit+1, convert_frac=peak_frac_bit)

out_path = './json_output/'
out_file_name = "spectra_fp16_mz_{}_{}_peak_{}_{}".format(mz_exp_bit, mz_frac_bit, peak_exp_bit, peak_frac_bit)
out_file_name = out_path + out_file_name

args = {
    'file_list': mgf_file_list, 
    'mz_min': 200, 
    'mz_max': 2000, 
    'min_mz_range': 250, 
    'min_peaks': 5, 
    'max_peaks_used': 50, 
    'remove_precursor_tol': 1.5, 
    'min_intensity': 0.01, 
    'scaling': 'log', 
    'out_file_name': out_file_name,

    'if_fp_quantize': True,
    'mz_exp_bit': mz_exp_bit,
    'mz_frac_bit': mz_frac_bit,
    'peak_exp_bit': peak_exp_bit,
    'peak_frac_bit': peak_frac_bit,
    'fp_quantizer_mz': fp16_quantizer_mz,
    'fp_quantizer_peak': fp16_quantizer_peak
}

processed_spectra_dict = run_baseline(**args)

2021-07-14 12:39:11,074 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : Start processsing spectra
2021-07-14 12:39:11,074 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : Start processsing spectra
2021-07-14 12:39:11,074 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : Start processsing spectra
2021-07-14 12:39:11,074 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : Start processsing spectra


{'file_list': ['/home/user/bio/ms-dataset/PXD001197/OrbiElite04549.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04786.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04785.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04784.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04783.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04782.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04781.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04780.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04779.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04778.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04557.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04556.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04555.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04554.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04553.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04552.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04551.mgf',

2021-07-14 12:48:34,361 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : End processsing spectra
2021-07-14 12:48:34,361 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : End processsing spectra
2021-07-14 12:48:34,361 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : End processsing spectra
2021-07-14 12:48:34,361 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : End processsing spectra
2021-07-14 12:48:34,363 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : Merging spectra results
2021-07-14 12:48:34,363 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : Merging spectra results
2021-07-14 12:48:34,363 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : Merging spectra results
2021-07-14 12:48:34,363 INFO [ms_preprocess/MainProcess] <ipython-input-17-b292dd0ae1aa>.run_baseline : 

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  9.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  9.4min finished


--- Logging error ---
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.8/logging/__init__.py", line 1081, in emit
    msg = self.format(record)
  File "/home/user/anaconda3/lib/python3.8/logging/__init__.py", line 925, in format
    return fmt.format(record)
  File "/home/user/anaconda3/lib/python3.8/logging/__init__.py", line 664, in format
    record.message = record.getMessage()
  File "/home/user/anaconda3/lib/python3.8/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/user/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/user/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/user/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/user/anaconda

In [24]:
# Using standard bfp16 quantization
mz_exp_bit = 5
peak_exp_bit = 5

mz_frac_bit = 15-mz_exp_bit
peak_frac_bit = 15-peak_exp_bit

fp16_quantizer_mz = ms_quantizer(convert_exp=mz_exp_bit, convert_frac=mz_frac_bit)
fp16_quantizer_peak = ms_quantizer(convert_exp=peak_exp_bit, convert_frac=peak_frac_bit)

out_path = './json_output/'
out_file_name = "spectra_bfp16"
out_file_name = out_path + out_file_name

args = {
    'file_list': mgf_file_list, 
    'mz_min': 200, 
    'mz_max': 2000, 
    'min_mz_range': 250, 
    'min_peaks': 5, 
    'max_peaks_used': 50, 
    'remove_precursor_tol': 1.5, 
    'min_intensity': 0.01, 
    'scaling': 'log', 
    'out_file_name': out_file_name,

    'if_fp_quantize': True,
    'mz_exp_bit': mz_exp_bit,
    'mz_frac_bit': mz_frac_bit,
    'peak_exp_bit': peak_exp_bit,
    'peak_frac_bit': peak_frac_bit,
    'fp_quantizer_mz': fp16_quantizer_mz,
    'fp_quantizer_peak': fp16_quantizer_peak
}

processed_spectra_dict = run_baseline(**args)

2021-07-14 13:03:49,053 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : Start processsing spectra
2021-07-14 13:03:49,053 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : Start processsing spectra
2021-07-14 13:03:49,053 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : Start processsing spectra
2021-07-14 13:03:49,053 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : Start processsing spectra


{'file_list': ['/home/user/bio/ms-dataset/PXD001197/OrbiElite04549.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04786.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04785.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04784.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04783.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04782.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04781.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04780.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04779.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04778.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04557.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04556.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04555.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04554.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04553.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04552.mgf', '/home/user/bio/ms-dataset/PXD001197/OrbiElite04551.mgf',

2021-07-14 13:13:16,269 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : End processsing spectra
2021-07-14 13:13:16,269 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : End processsing spectra
2021-07-14 13:13:16,269 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : End processsing spectra
2021-07-14 13:13:16,269 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : End processsing spectra
2021-07-14 13:13:16,271 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : Merging spectra results
2021-07-14 13:13:16,271 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : Merging spectra results
2021-07-14 13:13:16,271 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : Merging spectra results
2021-07-14 13:13:16,271 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  9.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  9.5min finished


2021-07-14 13:13:17,017 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : All finished! With quantization error: 19858.120434 35252417.435901
2021-07-14 13:13:17,017 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : All finished! With quantization error: 19858.120434 35252417.435901
2021-07-14 13:13:17,017 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : All finished! With quantization error: 19858.120434 35252417.435901
2021-07-14 13:13:17,017 INFO [ms_preprocess_fp/MainProcess] <ipython-input-23-5db1b96efb39>.run_baseline : All finished! With quantization error: 19858.120434 35252417.435901
